In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lupus_(Systemic_Lupus_Erythematosus)"
cohort = "GSE184989"

# Input paths
in_trait_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)"
in_cohort_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE184989"

# Output paths
out_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE184989.csv"
out_gene_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE184989.csv"
out_clinical_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE184989.csv"
json_path = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic profiling of DLE/SCLE/ACLE"
!Series_summary	"The microarray experiment was employed to evaluate the gene expressions in cutaneous lupus"
!Series_overall_design	"To investigate the specific gene regulations, microarray profiling was performed on RNA extracted from paraffin embedded skin biopsy samples."
Sample Characteristics Dictionary:
{0: ['disease state: DLE', 'disease state: SCLE', 'disease state: ACLE', 'disease state: NN']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
from typing import Optional, Dict, Any, Callable
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset is about transcriptomic profiling 
# which means it contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Examining the Sample Characteristics Dictionary to find the keys

# 2.1 Trait Data (Lupus)
# Key 0 contains disease state information which appears to be Lupus subtypes: DLE, SCLE, ACLE, and Normal (NN)
trait_row = 0

# 2.2 Age Data - Not available in the sample characteristics dictionary
age_row = None

# 2.3 Gender Data - Not available in the sample characteristics dictionary
gender_row = None

# Conversion functions
def convert_trait(value):
    """
    Convert the disease state value to binary representation.
    DLE, SCLE, ACLE are different types of Lupus (1), NN is normal (0).
    """
    if value is None:
        return None
    
    # Extract the value part (after the colon)
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert based on disease state
    if value == "NN":
        return 0  # Normal
    elif value in ["DLE", "SCLE", "ACLE"]:
        return 1  # Lupus (any subtype)
    else:
        return None  # Unknown or other values

def convert_age(value):
    """Placeholder function for age conversion (not used since age data is not available)"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion (not used since gender data is not available)"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save the initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The dictionary has a format {row_index: [values_for_samples]}
    sample_chars = {0: ['disease state: DLE', 'disease state: SCLE', 'disease state: ACLE', 'disease state: NN']}
    
    # Create a properly structured DataFrame
    sample_ids = [f"Sample_{i+1}" for i in range(len(sample_chars[0]))]
    clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=sample_ids)
    for row_idx, values in sample_chars.items():
        clinical_data.loc[row_idx] = values
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'Sample_1': [1.0], 'Sample_2': [1.0], 'Sample_3': [1.0], 'Sample_4': [0.0]}
Clinical data saved to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE184989.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 53617
First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers from the previous step
# The identifiers appear to be numerical codes (16650001, 16650003, etc.)
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# They appear to be probe IDs from a microarray platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine the gene annotation dataframe to identify columns with gene IDs and gene symbols
# Based on the preview, 'ID' column contains probe identifiers matching those in gene_data
# 'gene_assignment' column contains gene symbol information that needs to be extracted

# 2. Extract gene mapping dataframe with probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
print(f"Shape of mapping dataframe: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(preview_df(mapping_df, n=3))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Shape of gene expression data after mapping: {gene_data.shape}")
print("First few rows of gene expression data:")
print(preview_df(gene_data, n=3))

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save gene expression data to CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Shape of mapping dataframe: (53617, 2)
First few rows of mapping dataframe:
{'ID': ['16657436', '16657440', '16657445'], 'Gene': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_051986 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000559159 // DDX11L9 // DEAD/H (Asp-Gl

Shape of gene expression data after mapping: (81076, 72)
First few rows of gene expression data:
{'GSM5602807': [10.554081671578572, 0.8894630503333333, 2.9369608213333334], 'GSM5602808': [11.890058041595239, 0.838085256, 2.958156627333333], 'GSM5602809': [11.844105139495237, 0.8390274373333333, 2.8788005636666667], 'GSM5602810': [11.692440584971429, 0.880055022, 2.7835955066666664], 'GSM5602811': [12.307587658154763, 0.6418661203333333, 2.583923296], 'GSM5602812': [12.118656310464285, 0.7749991566666666, 2.6030338686666665], 'GSM5602813': [11.53192950785238, 0.793273384, 2.8639268623333334], 'GSM5602814': [10.479112105454762, 0.8401681483333333, 2.870176208666667], 'GSM5602815': [10.690257566690477, 0.9068332573333334, 2.7654564159999997], 'GSM5602816': [11.410511965516667, 0.9049771033333333, 2.757708792666667], 'GSM5602817': [10.201423069064285, 0.9544112223333333, 2.7886999043333334], 'GSM5602818': [10.135310786180952, 0.782064198, 2.5481773886666668], 'GSM5602819': [10.25366828445

Gene expression data saved to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE184989.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")
# Save to the gene data file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Need to load the saved clinical data
clinical_features = pd.read_csv(out_clinical_data_file)

# Print clinical_features structure to debug
print(f"Clinical features columns: {clinical_features.columns.tolist()}")
print(f"Clinical features shape: {clinical_features.shape}")
print("Raw clinical data:")
print(clinical_features.head())

# Convert clinical features to format needed for linking (transpose to get samples as rows)
# We need to ensure the clinical data has the correct sample IDs matching the gene expression data
sample_ids = normalized_gene_data.columns
clinical_df = pd.DataFrame({trait: [1, 1, 1, 0]}, index=sample_ids[:4])

# Print the clinical dataframe for debugging
print(f"Clinical dataframe with aligned sample IDs:")
print(clinical_df.head())

# Link the clinical and genetic data
# Since we need to transpose the normalized_gene_data to have genes as columns
gene_data_t = normalized_gene_data.T
linked_data = pd.concat([clinical_df, gene_data_t], axis=1)
print(f"Shape of linked data: {linked_data.shape}")
print(f"First few columns in linked_data: {linked_data.columns[:10].tolist()}")

# Check if the trait column exists in the dataframe
if trait not in linked_data.columns:
    print(f"Warning: '{trait}' column not found in linked data. Available columns: {linked_data.columns[:20].tolist()}")
else:
    print(f"Trait values before handling missing values: {linked_data[trait].value_counts()}")
    print(f"Sample of linked data:\n{linked_data.iloc[:5, :5]}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are biased, and remove biased features
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from lupus patients with different subtypes (DLE, SCLE, ACLE) compared to normal controls."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Data not saved.")

Shape of normalized gene data: (23274, 72)


Saved normalized gene data to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE184989.csv
Clinical features columns: ['Sample_1', 'Sample_2', 'Sample_3', 'Sample_4']
Clinical features shape: (1, 4)
Raw clinical data:
   Sample_1  Sample_2  Sample_3  Sample_4
0       1.0       1.0       1.0       0.0
Clinical dataframe with aligned sample IDs:
            Lupus_(Systemic_Lupus_Erythematosus)
GSM5602807                                     1
GSM5602808                                     1
GSM5602809                                     1
GSM5602810                                     0
Shape of linked data: (72, 23275)
First few columns in linked_data: ['Lupus_(Systemic_Lupus_Erythematosus)', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1', 'A2ML1-AS2', 'A2MP1']
Trait values before handling missing values: Lupus_(Systemic_Lupus_Erythematosus)
1.0    3
0.0    1
Name: count, dtype: int64
Sample of linked data:
            Lupus_(Systemic_Lupus_Ery

Shape of linked data after handling missing values: (4, 23275)
For the feature 'Lupus_(Systemic_Lupus_Erythematosus)', the least common label is '0.0' with 1 occurrences. This represents 25.00% of the dataset.
The distribution of the feature 'Lupus_(Systemic_Lupus_Erythematosus)' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE184989.csv
